In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import set_seed
import torch
from torch.optim import AdamW
from huggingface_hub import HfApi, login, hf_hub_download
from google.colab import userdata

from supplementary_file_for_sentence_similarity import *

# Loading the tokenizer and a dataset

In [2]:
# Loading the tokenizer

model_name = 'roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [3]:
dataloader = sentence_similarity_dataloaders(tokenizer)

set_seed(42)
train_dataloader, val_dataloader, test_dataloader = dataloader.get_dataloaders()

README.md: 0.00B [00:00, ?B/s]

mrpc/train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

mrpc/validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

mrpc/test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

# Training Experiments

In [ ]:
lr_list = [1e-5, 3e-5, 5e-5]
num_epochs = 3
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

### With Random seed: 137

In [ ]:
for lr in lr_list:

  for lr_scheduler in [False, True]:

    set_seed(137)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    trainer_config = {'optimizer' : AdamW,
                  'num_epochs' : num_epochs,
                  'learning_rate' : lr,
                  'lr_scheduler' : lr_scheduler,
                  }

    trainer = sentence_similarity_trainer(model=model,
                      train_dataloader = train_dataloader,
                      val_dataloader = val_dataloader,
                      device = device,
                      trainer_config = trainer_config,
                      )

    # Running the training loops
    print("="*20, f"{lr=} and {lr_scheduler = }", "="*20)
    trainer.train()

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


==================== lr=1e-05 and lr_scheduler = False ====================
Epoch 0 Step 0 -- training loss: 0.7095115031551951 --validation loss: 0.710458541617674 -- validation accuracy 0.3161764705882353
Epoch 0 Step 50 -- training loss: 0.6192328661095863 --validation loss: 0.6165644386235405 -- validation accuracy 0.6838235294117647
Epoch 0 Step 100 -- training loss: 0.6095815974253195 --validation loss: 0.6076498849719179 -- validation accuracy 0.6838235294117647
Epoch 0 Step 150 -- training loss: 0.6517966723714779 --validation loss: 0.6414962951459137 -- validation accuracy 0.6838235294117647
Epoch 0 Step 200 -- training loss: 0.5148398482721616 --validation loss: 0.5041248327961155 -- validation accuracy 0.7156862745098039
Epoch 0 Step 250 -- training loss: 0.43404882982340776 --validation loss: 0.4147685043075505 -- validation accuracy 0.7965686274509803
Epoch 0 Step 300 -- training loss: 0.4147768060155294 --validation loss: 0.3979383258550775 -- validation accuracy 0.806372

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 2 Step 458 -- training loss: 0.06188034750866117 --validation loss: 0.3518341007258962 -- validation accuracy 0.8872549019607843
The best accuracy was 0.8970588235294118 after step 458 of epoch 1.
==================== lr=1e-05 and lr_scheduler = True ====================
Epoch 0 Step 0 -- training loss: 0.7095115031551951 --validation loss: 0.710458541617674 -- validation accuracy 0.3161764705882353
Epoch 0 Step 50 -- training loss: 0.6150312169185131 --validation loss: 0.6097743581323063 -- validation accuracy 0.6838235294117647
Epoch 0 Step 100 -- training loss: 0.6003719088428681 --validation loss: 0.5944801551454207 -- validation accuracy 0.6838235294117647
Epoch 0 Step 150 -- training loss: 0.6197447857955443 --validation loss: 0.6041837822572858 -- validation accuracy 0.6838235294117647
Epoch 0 Step 200 -- training loss: 0.4983413711449939 --validation loss: 0.4885190509113611 -- validation accuracy 0.7205882352941176
Epoch 0 Step 250 -- training loss: 0.4058675510090551 --

...

Epoch 2 Step 458 -- training loss: 0.1632902209823525 --validation loss: 0.3680057324761269 -- validation accuracy 0.8553921568627451
The best accuracy was 0.8553921568627451 after step 300 of epoch 2.
==================== lr=3e-05 and lr_scheduler = True ====================
Epoch 0 Step 0 -- training loss: 0.6947074596139081 --validation loss: 0.6949678577628791 -- validation accuracy 0.37254901960784315
Epoch 0 Step 50 -- training loss: 0.617431222223768 --validation loss: 0.6131184200445811 -- validation accuracy 0.6985294117647058
Epoch 0 Step 100 -- training loss: 0.6085776200351632 --validation loss: 0.6060074690510245 -- validation accuracy 0.7107843137254902
Epoch 0 Step 150 -- training loss: 0.5272546563143304 --validation loss: 0.5223024668646794 -- validation accuracy 0.7303921568627451
Epoch 0 Step 200 -- training loss: 0.48064474761486053 --validation loss: 0.4595169302295236 -- validation accuracy 0.7254901960784313
Epoch 0 Step 250 -- training loss: 0.5003743044756077 -

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 2 Step 458 -- training loss: 0.07652352147782322 --validation loss: 0.3065065922599067 -- validation accuracy 0.8872549019607843
The best accuracy was 0.9068627450980392 after step 458 of epoch 1.
==================== lr=5e-05 and lr_scheduler = False ====================
Epoch 0 Step 0 -- training loss: 0.6810652118882322 --validation loss: 0.6807034506517298 -- validation accuracy 0.6838235294117647
Epoch 0 Step 50 -- training loss: 0.6291330246364369 --validation loss: 0.6241140336382622 -- validation accuracy 0.7009803921568627
Epoch 0 Step 100 -- training loss: 0.63473338576963 --validation loss: 0.6302839213726568 -- validation accuracy 0.6838235294117647
Epoch 0 Step 150 -- training loss: 0.6711580652686765 --validation loss: 0.6587418905075859 -- validation accuracy 0.6838235294117647
Epoch 0 Step 200 -- training loss: 0.6330896982829815 --validation loss: 0.627539265389536 -- validation accuracy 0.6838235294117647
Epoch 0 Step 250 -- training loss: 0.6306621868625965 --v

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 2 Step 458 -- training loss: 0.6307238418941664 --validation loss: 0.6241848164913701 -- validation accuracy 0.6838235294117647
The best accuracy was 0.7009803921568627 after step 50 of epoch 0.
==================== lr=5e-05 and lr_scheduler = True ====================
Epoch 0 Step 0 -- training loss: 0.6810652118882322 --validation loss: 0.6807034506517298 -- validation accuracy 0.6838235294117647
Epoch 0 Step 50 -- training loss: 0.6321441878550452 --validation loss: 0.63021014250961 -- validation accuracy 0.7156862745098039
Epoch 0 Step 100 -- training loss: 0.6123922724349826 --validation loss: 0.6085916097257652 -- validation accuracy 0.6838235294117647
Epoch 0 Step 150 -- training loss: 0.5726706071357062 --validation loss: 0.5737485938212451 -- validation accuracy 0.6838235294117647
Epoch 0 Step 200 -- training loss: 0.6193472639148272 --validation loss: 0.615129151765038 -- validation accuracy 0.6838235294117647
Epoch 0 Step 250 -- training loss: 0.6301382742416365 --vali

**Observations**:

In experiments 1-4, we see overfitting after epoch 1. So we ignore epoch 2 in these experiments.

In experiments 1,2, and 4, we also see some oscillations in both train and validation losses in epoch 1, but overall losses go down. In experiment 3, we see oscillations in both losses even from the beginning.

The best results for experiment 1 is achieved after step 458 of epoch 1, where the validation accuracy and validation loss are 89.7 and 0.280, respectively.

The best results for experiment 2 is achieved after step 458 of epoch 1, where the validation accuracy and validation loss are 89.0 and 0.282, respectively.

The best results for experiment 3 is achieved after step 450 of epoch 1, where the validation accuracy and validation loss are 81.4 and 0.385, respectively.

The best results for experiment 4 is achieved after step 458 of epoch 1, where the validation accuracy and validation loss are 90.7 and 0.272, respectively.

In experiments 5 and 6, both train and validation losses remain constant (up to some oscillations) and the accuracy remain constant as well. This suggests that the (consntant or initial) learning rate of $5\times 10^{-5}$ is probably too large.

Based on these observations, we conclude that experiment 4 leads to the best results.

### With Random seed: 23

In [ ]:
for lr in lr_list:

  for lr_scheduler in [False, True]:

    set_seed(23)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    trainer_config = {'optimizer' : AdamW,
                  'num_epochs' : num_epochs,
                  'learning_rate' : lr,
                  'lr_scheduler' : lr_scheduler,
                  }

    trainer = sentence_similarity_trainer(model=model,
                      train_dataloader = train_dataloader,
                      val_dataloader = val_dataloader,
                      device = device,
                      trainer_config = trainer_config,
                      )

    # Running the training loops
    print("="*20, f"{lr=} and {lr_scheduler = }", "="*20)
    trainer.train()

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


==================== lr=1e-05 and lr_scheduler = False ====================
Epoch 0 Step 0 -- training loss: 0.6695231836865411 --validation loss: 0.6682174696641809 -- validation accuracy 0.6838235294117647
Epoch 0 Step 50 -- training loss: 0.5935074584416574 --validation loss: 0.5881349900189567 -- validation accuracy 0.6838235294117647
Epoch 0 Step 100 -- training loss: 0.5653804466615315 --validation loss: 0.5531215416450127 -- validation accuracy 0.6838235294117647
Epoch 0 Step 150 -- training loss: 0.5381281334979862 --validation loss: 0.5281262800974005 -- validation accuracy 0.7450980392156863
Epoch 0 Step 200 -- training loss: 0.42328779250058735 --validation loss: 0.4217899751429464 -- validation accuracy 0.8284313725490197
Epoch 0 Step 250 -- training loss: 0.3967063057851168 --validation loss: 0.4040442792808308 -- validation accuracy 0.8161764705882353
Epoch 0 Step 300 -- training loss: 0.43097002489374614 --validation loss: 0.4330519204046212 -- validation accuracy 0.7965

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


==================== lr=1e-05 and lr_scheduler = True ====================
Epoch 0 Step 0 -- training loss: 0.6695231836865411 --validation loss: 0.6682174696641809 -- validation accuracy 0.6838235294117647
Epoch 0 Step 50 -- training loss: 0.6069191875411015 --validation loss: 0.600809653600057 -- validation accuracy 0.6838235294117647
Epoch 0 Step 100 -- training loss: 0.5731071185052784 --validation loss: 0.564489521816665 -- validation accuracy 0.6838235294117647
Epoch 0 Step 150 -- training loss: 0.5374747838719478 --validation loss: 0.5288021950160756 -- validation accuracy 0.6985294117647058
Epoch 0 Step 200 -- training loss: 0.4268347047642685 --validation loss: 0.41702817584954055 -- validation accuracy 0.8161764705882353
Epoch 0 Step 250 -- training loss: 0.39263811452651076 --validation loss: 0.39190588248711006 -- validation accuracy 0.8529411764705882
Epoch 0 Step 300 -- training loss: 0.3814238416343473 --validation loss: 0.37863950475173835 -- validation accuracy 0.85049

...

Epoch 2 Step 458 -- training loss: 0.06764359988905246 --validation loss: 0.3385030342740755 -- validation accuracy 0.8725490196078431
The best accuracy was 0.8995098039215687 after step 458 of epoch 1.
==================== lr=3e-05 and lr_scheduler = True ====================
Epoch 0 Step 0 -- training loss: 0.6589438468542493 --validation loss: 0.6570845316438114 -- validation accuracy 0.6838235294117647
Epoch 0 Step 50 -- training loss: 0.5863408955353797 --validation loss: 0.5830099249587339 -- validation accuracy 0.7181372549019608
Epoch 0 Step 100 -- training loss: 0.5481619011683792 --validation loss: 0.5280616002924302 -- validation accuracy 0.7377450980392157
Epoch 0 Step 150 -- training loss: 0.6537124352257756 --validation loss: 0.6515997765110988 -- validation accuracy 0.6862745098039216
Epoch 0 Step 200 -- training loss: 0.4394521601579288 --validation loss: 0.4309568892360902 -- validation accuracy 0.8186274509803921
Epoch 0 Step 250 -- training loss: 0.412158476923286 --

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


==================== lr=5e-05 and lr_scheduler = False ====================
Epoch 0 Step 0 -- training loss: 0.6497920916490825 --validation loss: 0.6472402086444929 -- validation accuracy 0.6838235294117647
Epoch 0 Step 50 -- training loss: 0.633785577186572 --validation loss: 0.6290013357704761 -- validation accuracy 0.6838235294117647
Epoch 0 Step 100 -- training loss: 0.6356001748238773 --validation loss: 0.627111950341393 -- validation accuracy 0.6838235294117647
Epoch 0 Step 150 -- training loss: 0.632310843220983 --validation loss: 0.6266066712491652 -- validation accuracy 0.6838235294117647
Epoch 0 Step 200 -- training loss: 0.6412640421738551 --validation loss: 0.6374463427300546 -- validation accuracy 0.6838235294117647
Epoch 0 Step 250 -- training loss: 0.6424187903570454 --validation loss: 0.6323590138379265 -- validation accuracy 0.6838235294117647
Epoch 0 Step 300 -- training loss: 0.6316689267787019 --validation loss: 0.6242588103986254 -- validation accuracy 0.683823529

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


==================== lr=5e-05 and lr_scheduler = True ====================
Epoch 0 Step 0 -- training loss: 0.6497920916490825 --validation loss: 0.6472402086444929 -- validation accuracy 0.6838235294117647
Epoch 0 Step 50 -- training loss: 0.6328684729818166 --validation loss: 0.6276626131113838 -- validation accuracy 0.6838235294117647
Epoch 0 Step 100 -- training loss: 0.63088196243336 --validation loss: 0.6246747321942273 -- validation accuracy 0.6838235294117647
Epoch 0 Step 150 -- training loss: 0.632264292123271 --validation loss: 0.6265431017267937 -- validation accuracy 0.6838235294117647
Epoch 0 Step 200 -- training loss: 0.6398025857597135 --validation loss: 0.6357908903383741 -- validation accuracy 0.6838235294117647
Epoch 0 Step 250 -- training loss: 0.6387009465486655 --validation loss: 0.6292142546644398 -- validation accuracy 0.6838235294117647
Epoch 0 Step 300 -- training loss: 0.630721917923759 --validation loss: 0.6240334563395556 -- validation accuracy 0.68382352941

**Observations**:

In experiments 1-4, we see overfitting after epoch 1. Even though we see some oscillations in train and validation losses in epoch 1 of these experiments, overall losses go down.

The best results for experiment 1 is achieved after step 450 of epoch 1, where the validation accuracy and validation loss are 88.2 and 0.289, respectively.

The best results for experiment 2 is achieved after step 50 of epoch 2, where the validation accuracy and validation loss are 89.2 and 0.279, respectively.

The best results for experiment 3 is achieved after step 458 of epoch 1, where the validation accuracy and validation loss are 90.0 and 0.267, respectively.

The best results for experiment 4 is achieved after step 458 of epoch 1, where the validation accuracy and validation loss are 88.5 and 0.258, respectively.

In experiments 5 and 6, train and validation losses (as well as validation accuracy) remain constant.

Based on these observations, both experiments 3 and 4 seem to give us the best results. Experiment 3 leads to the best validation accuracy (90.0) whereas experiment 4 leads a better validation loss (0.258). We choose to prefer lower validation loss than higher validation accuracy, and for this reason, we take experiment 4 as a better option.

### With Random seed: 114

In [ ]:
for lr in lr_list:

  for lr_scheduler in [False, True]:

    set_seed(114)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    trainer_config = {'optimizer' : AdamW,
                  'num_epochs' : num_epochs,
                  'learning_rate' : lr,
                  'lr_scheduler' : lr_scheduler,
                  }

    trainer = sentence_similarity_trainer(model=model,
                      train_dataloader = train_dataloader,
                      val_dataloader = val_dataloader,
                      device = device,
                      trainer_config = trainer_config,
                      )

    # Running the training loops
    print("="*20, f"{lr=} and {lr_scheduler = }", "="*20)
    trainer.train()

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


==================== lr=1e-05 and lr_scheduler = False ====================
Epoch 0 Step 0 -- training loss: 0.6472485588786389 --validation loss: 0.6439174331870734 -- validation accuracy 0.6838235294117647
Epoch 0 Step 50 -- training loss: 0.6174490010946145 --validation loss: 0.6144932327317256 -- validation accuracy 0.6838235294117647
Epoch 0 Step 100 -- training loss: 0.5673886464831616 --validation loss: 0.5620796084403992 -- validation accuracy 0.6838235294117647
Epoch 0 Step 150 -- training loss: 0.46169380928858433 --validation loss: 0.4546802821112614 -- validation accuracy 0.75
Epoch 0 Step 200 -- training loss: 0.4527818841417371 --validation loss: 0.4490890573052799 -- validation accuracy 0.7720588235294118
Epoch 0 Step 250 -- training loss: 0.38337334729974565 --validation loss: 0.3824194018162933 -- validation accuracy 0.8431372549019608
Epoch 0 Step 300 -- training loss: 0.37189484291054586 --validation loss: 0.37868840539572285 -- validation accuracy 0.8480392156862745

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


==================== lr=1e-05 and lr_scheduler = True ====================
Epoch 0 Step 0 -- training loss: 0.6472485588786389 --validation loss: 0.6439174331870734 -- validation accuracy 0.6838235294117647
Epoch 0 Step 50 -- training loss: 0.6222130482882456 --validation loss: 0.6181539974960626 -- validation accuracy 0.6838235294117647
Epoch 0 Step 100 -- training loss: 0.5920884793620224 --validation loss: 0.5875646337574604 -- validation accuracy 0.6838235294117647
Epoch 0 Step 150 -- training loss: 0.4615751320013293 --validation loss: 0.47049711337860894 -- validation accuracy 0.7475490196078431
Epoch 0 Step 200 -- training loss: 0.43364473028120654 --validation loss: 0.43219192559812586 -- validation accuracy 0.7230392156862745
Epoch 0 Step 250 -- training loss: 0.3719698004505733 --validation loss: 0.3832336437030166 -- validation accuracy 0.8406862745098039
Epoch 0 Step 300 -- training loss: 0.4423681117740332 --validation loss: 0.44244082213616837 -- validation accuracy 0.789

...

==================== lr=3e-05 and lr_scheduler = True ====================
Epoch 0 Step 0 -- training loss: 0.6448946821144204 --validation loss: 0.6412843909918093 -- validation accuracy 0.6838235294117647
Epoch 0 Step 50 -- training loss: 0.622123702354161 --validation loss: 0.6173617606069527 -- validation accuracy 0.6838235294117647
Epoch 0 Step 100 -- training loss: 0.6766177692444496 --validation loss: 0.6768086646117416 -- validation accuracy 0.6985294117647058
Epoch 0 Step 150 -- training loss: 0.5808221340049585 --validation loss: 0.5813268896411447 -- validation accuracy 0.7058823529411765
Epoch 0 Step 200 -- training loss: 0.4874726285374762 --validation loss: 0.4773085576062109 -- validation accuracy 0.6911764705882353
Epoch 0 Step 250 -- training loss: 0.41652365569390504 --validation loss: 0.4409679989604389 -- validation accuracy 0.8137254901960784
Epoch 0 Step 300 -- training loss: 0.4179290009739612 --validation loss: 0.41499631399033116 -- validation accuracy 0.843137

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


==================== lr=5e-05 and lr_scheduler = False ====================
Epoch 0 Step 0 -- training loss: 0.6426618735514954 --validation loss: 0.638711319250219 -- validation accuracy 0.6838235294117647
Epoch 0 Step 50 -- training loss: 0.6316457829703952 --validation loss: 0.6257051159353817 -- validation accuracy 0.6838235294117647
Epoch 0 Step 100 -- training loss: 0.6337695747678835 --validation loss: 0.6285211823734582 -- validation accuracy 0.6838235294117647
Epoch 0 Step 150 -- training loss: 0.6347753252988287 --validation loss: 0.6299369790974785 -- validation accuracy 0.6838235294117647
Epoch 0 Step 200 -- training loss: 0.6383528900302313 --validation loss: 0.629155566294988 -- validation accuracy 0.6838235294117647
Epoch 0 Step 250 -- training loss: 0.6334736769235731 --validation loss: 0.6251174398497039 -- validation accuracy 0.6838235294117647
Epoch 0 Step 300 -- training loss: 0.6321934622617069 --validation loss: 0.6241308170206407 -- validation accuracy 0.68382352

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


==================== lr=5e-05 and lr_scheduler = True ====================
Epoch 0 Step 0 -- training loss: 0.6426618735514954 --validation loss: 0.638711319250219 -- validation accuracy 0.6838235294117647
Epoch 0 Step 50 -- training loss: 0.6315188698779004 --validation loss: 0.6255039093541164 -- validation accuracy 0.6838235294117647
Epoch 0 Step 100 -- training loss: 0.6330436341513216 --validation loss: 0.6275956017129561 -- validation accuracy 0.6838235294117647
Epoch 0 Step 150 -- training loss: 0.6340110010441092 --validation loss: 0.6290045713677126 -- validation accuracy 0.6838235294117647
Epoch 0 Step 200 -- training loss: 0.6375863454051007 --validation loss: 0.6285182260999492 -- validation accuracy 0.6838235294117647
Epoch 0 Step 250 -- training loss: 0.6336133347617255 --validation loss: 0.6252190686908423 -- validation accuracy 0.6838235294117647
Epoch 0 Step 300 -- training loss: 0.6330326032015233 --validation loss: 0.6246570687667996 -- validation accuracy 0.68382352

**Observations**:

In experiments 1, 2, and 4, we see overfitting after epoch 1. In experiments 1,2, and 4, we also see some oscillations in both train and validation losses in epoch 1, but overall losses go down. In experiment 3, we see oscillations in both losses even in epoch 0, but overall losses go down until step 150 of epoch 2.

The best results for experiment 1 is achieved after step 450 of epoch 1, where the validation accuracy and validation loss are 88.7 and 0.288, respectively.

The best results for experiment 2 is achieved after step 458 of epoch 1, where the validation accuracy and validation loss are 88.5 and 0.289, respectively.

The best results for experiment 3 is achieved after step 150 of epoch 2, where the validation accuracy and validation loss are 88.5 and 0.278, respectively.

The best results for experiment 4 is achieved after step 458 of epoch 1, where the validation accuracy and validation loss are 86.8 and 0.298, respectively.

In experiments 5 and 6, both train and validation losses remain constant (up to some oscillations) and the accuracy remain constant as well. This suggests that the (consntant or initial) learning rate of $5\times 10^{-5}$ is probably too large.

Based on these observations, we conclude that experiment 3 leads to the best results.

# Conclusion

We conclude that the experiment with a random seed of 23 with a linearly decreasing learning rate from $3\times 10^{-5}$ to zero leads to the best result (validation loss of 0.258) if we stop after step 458 of epoch 1.

In [4]:
'''
Repeating the best performing experiment
with the stopping condition to get the
model weights and to calculate test accuracy.
'''

lr = 3e-5
lr_scheduler = True
num_epochs = 3
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
stopping_condition = {'step': 458, 'epoch': 1}

set_seed(23)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

trainer_config = {'optimizer' : AdamW,
              'num_epochs' : num_epochs,
              'learning_rate' : lr,
              'lr_scheduler' : lr_scheduler,
              }



trainer = sentence_similarity_trainer(model=model,
                  train_dataloader = train_dataloader,
                  val_dataloader = val_dataloader,
                  device = device,
                  trainer_config = trainer_config,
                  stopping_condition = stopping_condition,
                  )

# Running the training loops
trainer.train()

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 0 Step 0 -- training loss: 0.6589438468542493 --validation loss: 0.6570845316438114 -- validation accuracy 0.6838235294117647
Epoch 0 Step 50 -- training loss: 0.5863408955353797 --validation loss: 0.5830099249587339 -- validation accuracy 0.7181372549019608
Epoch 0 Step 100 -- training loss: 0.5481619011683792 --validation loss: 0.5280616002924302 -- validation accuracy 0.7377450980392157
Epoch 0 Step 150 -- training loss: 0.6537124352257756 --validation loss: 0.6515997765110988 -- validation accuracy 0.6862745098039216
Epoch 0 Step 200 -- training loss: 0.4394521601579288 --validation loss: 0.4309568892360902 -- validation accuracy 0.8186274509803921
Epoch 0 Step 250 -- training loss: 0.412158476923286 --validation loss: 0.39726442931329503 -- validation accuracy 0.8357843137254902
Epoch 0 Step 300 -- training loss: 0.43370816077477015 --validation loss: 0.4352886171317568 -- validation accuracy 0.8063725490196079
Epoch 0 Step 350 -- training loss: 0.38677314097520316 --validat

In [5]:
'''
Calculating the test loss
'''

def test_evaluation():

    trainer.model.eval()
    with torch.no_grad():

      test_losses = []
      test_accuracies = []

      for i, batch in enumerate(test_dataloader):

        # Getting the batch loss
        batch = {k: v.to(trainer.device) for k, v in batch.items()}
        outputs = trainer.model(**batch)
        test_losses.append(outputs.loss.item())
        # Getting the batch accuracy
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        test_accuracy = (predictions == batch['labels']).float().mean()
        test_accuracies.append(test_accuracy.item())

      avg_test_loss = sum(test_losses) / len(test_losses)
      avg_test_accuracy = sum(test_accuracies) / len(test_accuracies)

    trainer.model.train()

    return avg_test_loss, avg_test_accuracy

test_loss, test_acc = test_evaluation()
print(f"{test_loss=}")
print(f"{test_acc=}")

test_loss=0.3181992418184463
test_acc=0.8674768518518519


In [6]:
'''
Uploading the weights on HF
'''

# Save model weights
file_name = "model_weights.pth"
trainer.save_model(file_name)

# Logging into Hugging face Hub
hf_token = userdata.get('hf_TOKEN')
login(token=hf_token)
api = HfApi()

repo_id = "mudassirmoosa/sentence-similarity-transformer-comparison"

# Uploading model weights
api.upload_file(
    path_or_fileobj=file_name,
    path_in_repo="RoBERTa_for_sentence_similarity.pth",
    repo_id=repo_id,
    token=hf_token
)


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  model_weights.pth                     :   0%|          |  566kB /  499MB            

CommitInfo(commit_url='https://huggingface.co/mudassirmoosa/sentence-similarity-transformer-comparison/commit/ba70e57022248fadc243e4dbeaa1e4a6b4724724', commit_message='Upload RoBERTa_for_sentence_similarity.pth with huggingface_hub', commit_description='', oid='ba70e57022248fadc243e4dbeaa1e4a6b4724724', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mudassirmoosa/sentence-similarity-transformer-comparison', endpoint='https://huggingface.co', repo_type='model', repo_id='mudassirmoosa/sentence-similarity-transformer-comparison'), pr_revision=None, pr_num=None)